In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
gpt4o = pd.read_csv('/content/gdrive/MyDrive/gpt4o_generated_text_2405.csv')

In [ ]:
gpt4o.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419 entries, 0 to 418
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   text             419 non-null    object 
 1   label            419 non-null    object 
 2   prompt_n         419 non-null    int64  
 3   PredictedLabel   419 non-null    object 
 4   ConfidenceScore  419 non-null    float64
 5   gpt4_label       419 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 19.8+ KB


In [ ]:
gpt4o = gpt4o[['text','label']].copy()
gpt4o.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419 entries, 0 to 418
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    419 non-null    object
 1   label   419 non-null    object
dtypes: object(2)
memory usage: 6.7+ KB


In [ ]:
!pip install openai
from openai import OpenAI
client = OpenAI(api_key='sk-proj-hEv32vgqXTnk8oxTyhhpT3BlbkFJSBixmeuoz6s5mkpOg9NU',)
import pandas as pd
import re  # Import the regex module

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00


In [ ]:
def create_merged_texts(gpt4):
  data = gpt4['text'].tolist()  # Convert the DataFrame column to a list for easier manipulation

  merged_texts = []  # This will store all merged text blocks

  # Process each block of 100 statements
  for i in range(0, len(data), 100):
      # Extract the slice for the current block
      block = data[i:i+100]
      # Merge the statements with numbering and new lines
      merged_text = '\n'.join(f"{j+1}. {s}" for j, s in enumerate(block))
      # Store the merged text in the list
      merged_texts.append(merged_text)

  return merged_texts


In [ ]:
def predict_financial_sentiment(user_prompt, max_tokens):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": "As a financial expert, analyze the potential immediate impact of financial news statements on the stock prices of the relevant companies. Assess whether each statement suggests a positive, negative, or neutral effect on the company’s stock price. Ensure that your analysis of each statement is conducted independently to avoid bias from other statements."
            },
            {
                "role": "user",
                "content": user_prompt
            }
        ],
        temperature=0.0, # Setting temperature to 0 for more deterministic outputs
        max_tokens=max_tokens,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    print(response)

    text_response = response.choices[0].message.content
    sentences = re.split(r'\d+\.\s', text_response)
    if sentences and sentences[0] == '':
        sentences = sentences[1:]

    # Strip leading and trailing whitespace and newline characters from each sentence
    sentences = [sentence.strip() for sentence in sentences]

    dataset = pd.DataFrame({
        'gpt4_label': sentences
    })

    return dataset

In [ ]:
# Initialize an empty list to store DataFrames
def create_label(merged_texts):
  results = []

  for statement in merged_texts:
      user_prompt = f"Acting as a financial expert, analyze the following 100 numbered independent financial news statements, each pertaining to a different company. Assess the potential immediate impact on the stock price of the relevant company independently of any other statements. Determine whether the statement suggests a positive, negative, or neutral immediate effect on the company’s stock price. Consider factors such as tone, relevance of information, and market context in your evaluation. Please only provide potential immediate impacts (positive, negative, or neutral) for each of the statements in a numbered format, with each prediction corresponding to the order of the statements provided.\n{statement}"
      try:
          # Append the result of each prediction to the results list
          results.append(predict_financial_sentiment(user_prompt, 600))
      except Exception as e:
          print(f"Error processing statement: {statement}\nError: {e}")

  return results



In [ ]:
# Shuffle the DataFrame rows
gpt4o = gpt4o.sample(frac=1).reset_index(drop=True)
merged_texts = create_merged_texts(gpt4o)
results = create_label(merged_texts)
df = pd.concat(results, ignore_index=True)
df.loc[:, 'gpt4_label'] = df['gpt4_label'].str.lower()
df = df.rename(columns={'gpt4_label': 'gpt4_label_1'})
# Combine the original data with the predictions
gpt4o = pd.concat([gpt4o,df.reset_index(drop=True)], axis=1)

ChatCompletion(id='chatcmpl-9SstW9omf8eW6H6mzGy2enEs2Vk3V', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='1. Neutral\n2. Neutral\n3. Neutral\n4. Neutral\n5. Neutral\n6. Neutral\n7. Neutral\n8. Neutral\n9. Neutral\n10. Neutral\n11. Neutral\n12. Neutral\n13. Neutral\n14. Neutral\n15. Neutral\n16. Neutral\n17. Negative\n18. Neutral\n19. Neutral\n20. Neutral\n21. Neutral\n22. Neutral\n23. Neutral\n24. Neutral\n25. Neutral\n26. Neutral\n27. Positive\n28. Neutral\n29. Negative\n30. Neutral\n31. Neutral\n32. Neutral\n33. Neutral\n34. Neutral\n35. Neutral\n36. Neutral\n37. Neutral\n38. Neutral\n39. Neutral\n40. Neutral\n41. Negative\n42. Neutral\n43. Neutral\n44. Neutral\n45. Negative\n46. Neutral\n47. Neutral\n48. Neutral\n49. Neutral\n50. Neutral\n51. Neutral\n52. Neutral\n53. Neutral\n54. Neutral\n55. Neutral\n56. Neutral\n57. Neutral\n58. Neutral\n59. Neutral\n60. Neutral\n61. Positive\n62. Neutral\n63. Neutral\n64. Neutral\n65. Neutra

In [ ]:
different_count = (gpt4o['gpt4_label_1'] != gpt4o['label']).sum()

print(f"Number of rows where gpt4_label1 differs from label: {different_count}")

Number of rows where gpt4_label1 differs from label: 5


In [ ]:
# Filter rows where 'label' and 'gpt4_label_1' match
gpt4o_filtered = gpt4o[(gpt4o['label'] == gpt4o['gpt4_label_1'])]

In [ ]:
# Shuffle the DataFrame rows
gpt4o_filtered = gpt4o_filtered.sample(frac=1).reset_index(drop=True)

merged_texts = create_merged_texts(gpt4o_filtered)
results = create_label(merged_texts)
df = pd.concat(results, ignore_index=True)
df.loc[:, 'gpt4_label'] = df['gpt4_label'].str.lower()
df = df.rename(columns={'gpt4_label': 'gpt4_label_2'})
# Combine the original data with the predictions
gpt4o_filtered = pd.concat([gpt4o_filtered,df.reset_index(drop=True)], axis=1)

ChatCompletion(id='chatcmpl-9SsuUDsGlGqwMKRTreXEKWR5CBWrq', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='1. Neutral\n2. Neutral\n3. Neutral\n4. Neutral\n5. Neutral\n6. Neutral\n7. Neutral\n8. Neutral\n9. Neutral\n10. Neutral\n11. Neutral\n12. Neutral\n13. Neutral\n14. Neutral\n15. Neutral\n16. Neutral\n17. Neutral\n18. Neutral\n19. Neutral\n20. Neutral\n21. Neutral\n22. Neutral\n23. Neutral\n24. Neutral\n25. Neutral\n26. Neutral\n27. Neutral\n28. Neutral\n29. Neutral\n30. Neutral\n31. Neutral\n32. Neutral\n33. Neutral\n34. Neutral\n35. Neutral\n36. Neutral\n37. Neutral\n38. Neutral\n39. Neutral\n40. Neutral\n41. Neutral\n42. Neutral\n43. Neutral\n44. Neutral\n45. Neutral\n46. Neutral\n47. Neutral\n48. Neutral\n49. Neutral\n50. Neutral\n51. Neutral\n52. Neutral\n53. Neutral\n54. Neutral\n55. Neutral\n56. Neutral\n57. Neutral\n58. Neutral\n59. Neutral\n60. Neutral\n61. Neutral\n62. Neutral\n63. Neutral\n64. Neutral\n65. Neutral\n66.

In [ ]:
different_count = (gpt4o_filtered['gpt4_label_2'] != gpt4o_filtered['label']).sum()

print(f"Number of rows where gpt4_label2 differs from label: {different_count}")

Number of rows where gpt4_label2 differs from label: 4


In [ ]:
# Filter rows where 'label' and 'gpt4_label_2' match
gpt4o_filtered = gpt4o_filtered[(gpt4o_filtered['label'] == gpt4o_filtered['gpt4_label_2'])]

In [ ]:
gpt4o_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 410 entries, 0 to 413
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    410 non-null    object
 1   label   410 non-null    object
dtypes: object(2)
memory usage: 9.6+ KB


In [ ]:
gpt4o_filtered = gpt4o_filtered.drop(columns=['gpt4_label_1', 'gpt4_label_2'])

In [ ]:
gpt4o_filtered.to_csv('/content/gdrive/MyDrive/gpt4o_generated_text_2405_v2.csv', index=False)